In [1]:
import pandas as pd


In [5]:
customer  = pd.read_csv("../data/customers 1.csv")
order_items = pd.read_csv("../data/order_items 1.csv")
orders = pd.read_csv("../data/orders 1.csv")
products = pd.read_csv("../data/products 1.csv")

In [20]:
sales = order_items.merge(orders,on="order_id",how="inner")

In [21]:
#Compute Line Total
sales['line_total'] = sales["price"] * sales['quantity']
sales.head(5)

,order_item_id,order_id,product_id,quantity,price,customer_id,order_date,status,line_total
0,1,1,P5,2,400,101,2023-08-01,CANCELLED,800
1,2,2,P5,3,200,103,2023-08-08,COMPLETE,600
2,3,2,P1,2,300,103,2023-08-08,COMPLETE,600
3,4,3,P2,1,400,102,2023-08-24,COMPLETE,400
4,5,4,P3,3,300,105,2023-08-03,CANCELLED,900


In [22]:
#Transformation 3: Filter Only Completed Orders
sales = sales[sales["status"]=="COMPLETE"]
sales.head(5)

,order_item_id,order_id,product_id,quantity,price,customer_id,order_date,status,line_total
1,2,2,P5,3,200,103,2023-08-08,COMPLETE,600
2,3,2,P1,2,300,103,2023-08-08,COMPLETE,600
3,4,3,P2,1,400,102,2023-08-24,COMPLETE,400
5,6,5,P2,3,100,104,2023-08-02,COMPLETE,300
6,7,5,P2,6,400,104,2023-08-02,COMPLETE,2400


In [29]:
import numpy as np
#Add Discount Column (Business Rule)
sales["discount"] = np.where(sales["quantity"] >=5,0.10* sales["line_total"],0)
sales["net_total"] = sales["line_total"] - (sales["discount"])

In [31]:
# 5. Derive Order Month & Year
# ------------------------------
sales['order_date'] = pd.to_datetime(sales['order_date'])
sales['order_year'] = sales['order_date'].dt.year
sales['order_month'] = sales['order_date'].dt.month

In [43]:
order_summary = sales.groupby('order_id').agg(
    order_total=('net_total', 'sum'),
    total_quantity=('quantity', 'sum'),
    customer_id=('customer_id', 'first'),
    order_year=('order_year', 'first'),
    order_month=('order_month', 'first')
).reset_index()

In [44]:
order_summary.head(5)

,order_id,order_total,total_quantity,customer_id,order_year,order_month
0,2,1200.0,5,103,2023,8
1,3,400.0,1,102,2023,8
2,5,2910.0,14,104,2023,8
3,6,2000.0,4,101,2023,8
4,7,1440.0,11,105,2023,8


In [45]:
# Transformation 7: Customer Region Join
order_summary = order_summary.merge(customer[["region","customer_id"]],on="customer_id",how= "left")
order_summary.head(5)

,order_id,order_total,total_quantity,customer_id,order_year,order_month,region
0,2,1200.0,5,103,2023,8,South
1,3,400.0,1,102,2023,8,West
2,5,2910.0,14,104,2023,8,North
3,6,2000.0,4,101,2023,8,East
4,7,1440.0,11,105,2023,8,East


In [46]:
# Transformation 8: Sales by Region + Month (Aggregation)
sales_region = order_summary.groupby(['region','order_year','order_month']).agg(
    Total_revenue=('order_total','sum'),
    Total_quantity = ('total_quantity','first')
).reset_index()

In [47]:
sales_region.head(5)

,region,order_year,order_month,Total_revenue,Total_quantity
0,East,2023,8,4040.0,4
1,North,2023,8,2910.0,14
2,South,2023,8,1200.0,5
3,West,2023,8,400.0,1


In [53]:
#Transformation 9: Category-wise Analysis (Enrichment)
sales_with_products = sales.merge(products[["product_id","category"]],on="product_id",how="left")
category_sales = sales_with_products.groupby("category").agg(
    category_revenue = ("net_total","sum"),
    category_order_count = ('order_id','sum')
).reset_index()

category_sales.head(5)

,category,category_revenue,category_order_count
0,Clothing,2000.0,12
1,Electronics,2950.0,25
2,Furniture,3600.0,29


In [ ]:
# Transformation 10: Ranking Transformation
# Example:  df['max_rank'] = df['Number_legs'].rank(method='max')
region_sales_ranked = sales_region.copy()
region_sales_ranked["Rank"] = region_sales_ranked.groupby(["order_month","order_year"])['Total_revenue'].rank(method='dense',ascending=False)
region_sales_ranked.head(5)

,region,order_year,order_month,Total_revenue,Total_quantity,Rank
0,East,2023,8,4040.0,4,1.0
1,North,2023,8,2910.0,14,2.0
2,South,2023,8,1200.0,5,3.0
3,West,2023,8,400.0,1,4.0


In [ ]:
#Transformation 11: Sorting values
region_sales_ranked.sort_values(by=['order_year','order_month','Total_revenue'],ascending= [True,True,False])

,region,order_year,order_month,Total_revenue,Total_quantity,Rank
0,East,2023,8,4040.0,4,1.0
1,North,2023,8,2910.0,14,2.0
2,South,2023,8,1200.0,5,3.0
3,West,2023,8,400.0,1,4.0


In [ ]:
    #Transformation 12: Outlier Flagging 
threshold = order_summary['order_total'].quantile(0.95)
order_summary['outlier_flag'] = np.where(order_summary['order_total'] > threshold, 1, 0)


In [73]:
from sqlalchemy import create_engine
import pandas as pd

# MySQL connection info
server = "localhost"     # MySQL host (can be IP or container name if Docker)
port = 3306              # Default MySQL port
database = "pythontraining"  # Database name
username = "root"
password = "NewStrong123!"

# SQLAlchemy connection string for MySQL (using pymysql)
connection_string = (
    f"mysql+pymysql://{username}:{password}@{server}:{port}/{database}"
)

# Create engine
engine = create_engine(connection_string)
df = pd.read_sql("SELECT * FROM category_sales;", engine)


In [74]:
df.head(5)


,category,category_revenue,category_order_count
0,Clothing,2000.0,12
1,Electronics,2950.0,25
2,Furniture,3600.0,29


In [65]:
order_summary.head(5)

,order_id,order_total,total_quantity,customer_id,order_year,order_month,region,outlier_flag
0,2,1200.0,5,103,2023,8,South,0
1,3,400.0,1,102,2023,8,West,0
2,5,2910.0,14,104,2023,8,North,1
3,6,2000.0,4,101,2023,8,East,0
4,7,1440.0,11,105,2023,8,East,0
